In [1]:
import hashlib
import random
import string

In [2]:
#Install numpy array library
!pip install numpy

In [3]:
#for performance test, time library will be used to compute duration
import time
#for performance test, numpy library will be used to store multiple experiment output as arrays
import numpy as np

Default Method

In [4]:
def find_leading_zeros_default(size):
    iterations=0
    pattern = "0"*size
    str_hex_chars=string.digits+"abcdef"
    while True:
        input_str = ''.join(random.choices(str_hex_chars,k=64))
        hash_str=hashlib.sha256(input_str.encode()).hexdigest()
        iterations+=1
        if hash_str[0:size]==pattern: 
            break

    return input_str,hash_str,iterations

Fast Method

In [5]:
def find_leading_zeros_fast(size):
    iterations=0
    pattern = "0"*size
    input_str = ''.join(random.choices(string.digits+"abcdef",k=64))

    while True:
        hash_str=hashlib.sha256(input_str.encode()).hexdigest()
        iterations+=1
        if hash_str[0:size]==pattern: 
            break
        input_str= hash_str

    return input_str,hash_str,iterations

#### Performance Test

In [6]:
#It executes the default or fast method of leading zeros 
#number of times mentioned by argument "exp_count" 
#with number of zeros mentioned by argument "size"
#"is_fast_method" =True means Fast method is called, otherwise "Default" method
#It returns duration array and iterations array containing data for multiple experiments 
def perf_test(exp_count,is_fast_method,size,is_print_detail=False):
    iterations_arr=np.zeros(exp_count,dtype=int)
    duration_arr=np.zeros(exp_count,dtype=int)

    for i in range(exp_count):
        start = time.time()
        #call default method or fast method based on the input argument
        if is_fast_method:
            matched_str,hash_str,iterations=find_leading_zeros_fast(size)
        else:
            matched_str,hash_str,iterations=find_leading_zeros_default(size)
        end = time.time()
        #note that time() method returns in seconds
        #so need to convert to milliseconds by multiplying with 1000
        duration = int((end-start)*1000)

        iterations_arr[i]=iterations
        duration_arr[i]=duration
        if is_print_detail:
            print(f"\n ************** i={i} ******************")
            print("Matched String : ",matched_str)
            print("Hash:            ",hash_str)
            print('Number of iterations to match:' ,iterations)
            print(f"Execution Time (in ms)      : {duration}")

    print("\n------------------------------------------")
    print("Experiment Results")
    print("------------------------------------------")
    print("Iterations:")
    print(iterations_arr)
    print("Durations (in ms):")
    print(duration_arr) 
    
    return iterations_arr,duration_arr

In [7]:
#number of zeros and number of experiments (no of tries) are specified below
perf_size=4
exp_count=50

#### Performance test of Default Method

In [8]:
iterations_def_arr,duration_def_arr=perf_test(exp_count,False,perf_size,is_print_detail=False)
#To view sha hash detail of each experiment, set the flag is_print_detail to True


------------------------------------------
Experiment Results
------------------------------------------
Iterations:
[160221  78686  33120   6689  33767  12659   4335  28340  35841  24853
  33456   2120  24902  89409 129583  96190  17946   3299  46020  16685
 104113  53520  51709   3864 223224  33898    846  41149  40177  23287
   3334   1388  50805 215845  67422  83746   8270  46439   4927  48887
  47899  28743  63401  43712  74044   7854  15402  85065 204660  93413]
Durations (in ms):
[4078 1995  845  194  856  335  109  765  912  627  849   53  630 2261
 3308 2434  453   83 1170  421 2634 1349 1325   97 5641  858   21 1039
 1013  593   96   48 1286 5524 1706 2138  209 1177  123 1234 1210  724
 1607 1105 1880  218  386 2143 5151 2384]


#### Performance test of Fast Method

In [9]:
iterations_fast_arr,duration_fast_arr=perf_test(exp_count,True,perf_size,is_print_detail=False)
#To view sha hash detail of each experiment, set the flag is_print_detail to True


------------------------------------------
Experiment Results
------------------------------------------
Iterations:
[ 59552  54801  33968  63883  89080  36912  53045  77966  27574 157214
  96895  42226  40898  26151   1288 147846  26146 177247  65783  98206
  60701  45522 150020   4269  20749  18579  50691   5039   3697  23485
  20144 112183 159481  35815  59297  41649  11481  95018  11677  56160
  13754  57914  97878  77005    144 101801  31534  12198   1165  86041]
Durations (in ms):
[119 107  65 126 170  72 105 151  53 308 186  82  80  51   2 291  50 348
 127 193 120  87 295   8  40  35  98   9   7  45  39 220 309  68 114  78
  22 203  37 143  27 132 203 147   0 194  60  23   2 165]


#### Display Chart to depict performance comparison between Default and Fast Method

In [10]:
#Install plotly chart library
!pip install plotly

In [11]:
import plotly.graph_objects as go

In [12]:
import plotly.io as pio
pio.renderers.default = 'iframe'

In [13]:
fig = go.Figure()

#add scatter plot for default method
#iterations_def_arr as x-axis data and
#duration_def_arr as y-axis data
fig.add_trace(go.Scatter(name="default", 
                            x=iterations_def_arr, 
                            y=duration_def_arr,
        fill=None,
        mode='markers',
        marker={'size':8,"symbol":"circle"},
        line_color='red',
      hovertemplate =
    '<i>Iterations</i>:    %{x:6d}'+
    '<br><b>Time in ms</b>:    %{y:.0f}<br>',))

#add scatter plot for fast method
#iterations_fast_arr as x-axis data and
#duration_fast_arr as y-axis data
fig.add_trace(go.Scatter(name="fast", 
                            x=iterations_fast_arr, 
                            y=duration_fast_arr, 
        fill=None,
        mode='markers',
        marker={'size':8,"symbol":"diamond"},
        line_color='yellow',
    hovertemplate =
    '<i>Iterations</i>:%{x:6d}'+
    '<br><b>Time in ms</b>: %{y:.0f}<br>',))
    

    
fig.update_layout({
        'autosize':False,
        'plot_bgcolor':'rgba(2,0,0,5)',
        'paper_bgcolor':'rgba(2,0,0,5)',
        'font_color':"white",
        'title': f'Performance Comparison (no of zeros={perf_size})',
        'yaxis_title': 'Time (in milliseconds)',
        'xaxis_title': 'Number of Iterations',
        'xaxis':{'showgrid':False},
        })
    
fig.update_yaxes(automargin = True, gridcolor = "gray")
fig.update_xaxes(automargin = True, gridcolor = "gray",showgrid=False)
   
fig.show()